---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [176]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    with open('university_towns.txt') as file:
        data = []
        for line in file:
            data.append(line[:-1])
    state_town = []
    for line in data:
        if line[-6:] == '[edit]':
            state = line[:-6]
        elif '(' in line:
            town = line[:line.index('(')-1]
            state_town.append([state,town])
        else:
            town = line
            state_town.append([state,town])
    state_college_df = pd.DataFrame(state_town,columns = ['State','RegionName'])
    return state_college_df
        

len(get_list_of_university_towns()['RegionName'].unique())

#x[x['State'] == 'Massachusetts']

489

In [35]:
def get_data():
    ColumnDict = {
        'Unnamed: 4': 'Quarter',
        'Unnamed: 0': 'Year'
    }
    recess_quarter = []
    excel_file = pd.ExcelFile('gdplev.xls')
    data_df = excel_file.parse('Sheet1', skiprows=5)
    data_df.reset_index(inplace=True)
    data_df.drop(labels=['Unnamed: 3', 'Unnamed: 7', 'index'], axis=1, inplace=True)
    data_df.drop([0,1], axis=0, inplace=True)
    data_df.rename(mapper=ColumnDict,axis=1, inplace=True)
    data_df.reset_index(inplace=True)
    data_df.drop('index', axis=1, inplace=True)
#     GDP = 'GDP in billions of chained 2009 dollars.1'
    GDP = 'GDP in billions of current dollars.1'
    data_df['Delta'] = data_df[GDP][1:].reset_index(drop=True)
    data_df = data_df[212:].reset_index()
    data_df['Delta'] = data_df['Delta'] - data_df[GDP]
    return data_df
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    ans = '2008q3'
    ColumnDict = {
        'Unnamed: 4': 'Quarter',
        'Unnamed: 0': 'Year'
    }
    recess_quarter = []
    excel_file = pd.ExcelFile('gdplev.xls')
    data_df = excel_file.parse('Sheet1', skiprows=5)
    data_df.reset_index(inplace=True)
    data_df.drop(labels=['Unnamed: 3', 'Unnamed: 7', 'index'], axis=1, inplace=True)
    data_df.drop([0,1], axis=0, inplace=True)
    data_df.rename(mapper=ColumnDict,axis=1, inplace=True)
    data_df.reset_index(inplace=True)
    data_df.drop('index', axis=1, inplace=True)
#     GDP = 'GDP in billions of chained 2009 dollars.1'
    GDP = 'GDP in billions of current dollars.1'
    data_df['Delta'] = data_df[GDP][1:].reset_index(drop=True)
    data_df = data_df[212:].reset_index()
    data_df['Delta'] = data_df['Delta'] - data_df[GDP]
    for i in range(len(data_df) - 1):
        if data_df.iloc[i]['Delta'] < 0 and data_df.iloc[i + 1]['Delta'] < 0:
            return data_df.iloc[i]['Quarter']
    return None
get_recession_start()

'2008q3'

In [42]:
get_data()[22:]

,index,Year,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,Quarter,GDP in billions of current dollars.1,GDP in billions of chained 2009 dollars.1,Delta
22,234,NaN,NaN,NaN,2005q3,13205.4,14291.8,176.2
23,235,NaN,NaN,NaN,2005q4,13381.6,14373.4,267.3
24,236,NaN,NaN,NaN,2006q1,13648.9,14546.1,150.9
25,237,NaN,NaN,NaN,2006q2,13799.8,14589.6,108.7
26,238,NaN,NaN,NaN,2006q3,13908.5,14602.6,157.9
27,239,NaN,NaN,NaN,2006q4,14066.4,14716.9,166.8
28,240,NaN,NaN,NaN,2007q1,14233.2,14726.0,189.1
29,241,NaN,NaN,NaN,2007q2,14422.3,14838.7,147.4
30,242,NaN,NaN,NaN,2007q3,14569.7,14938.5,115.6
31,243,NaN,NaN,NaN,2007q4,14685.3,14991.8,-16.9


In [34]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    ColumnDict = {
        'Unnamed: 4': 'Quarter',
        'Unnamed: 0': 'Year'
    }
    recess_quarter = []
    excel_file = pd.ExcelFile('gdplev.xls')
    data_df = excel_file.parse('Sheet1', skiprows=5)
    data_df.reset_index(inplace=True)
    data_df.drop(labels=['Unnamed: 3', 'Unnamed: 7', 'index'], axis=1, inplace=True)
    data_df.drop([0,1], axis=0, inplace=True)
    data_df.rename(mapper=ColumnDict,axis=1, inplace=True)
    data_df.reset_index(inplace=True)
    data_df.drop('index', axis=1, inplace=True)
#     GDP = 'GDP in billions of chained 2009 dollars.1'
    GDP = 'GDP in billions of current dollars.1'
    data_df['Delta'] = data_df[GDP][1:].reset_index(drop=True)
    data_df = data_df[212:].reset_index()
    data_df['Delta'] = data_df['Delta'] - data_df[GDP]
#     return data_df[29:]
    for i in range(len(data_df) - 1, 3 , -1):
        if data_df.iloc[i]['Delta'] > 0 and data_df.iloc[i - 1]['Delta'] > 0 and data_df.iloc[i -2]['Delta'] < 0 and data_df.iloc[i -3]['Delta'] < 0:
            return data_df.iloc[i + 1]['Quarter']
    ans = '2009q4'
    return data_df[29:]
get_recession_end()

'2009q4'

In [55]:
import numpy as np
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    data_df = get_data()
    bottoms = pd.Series()
    GDP = 'GDP in billions of current dollars.1'
    for i in range(len(data_df) - 2):
        if data_df.iloc[i]['Delta'] < 0 and data_df.iloc[i + 1]['Delta'] < 0:
            bottoms[data_df.iloc[i+1]['Quarter']] = data_df.iloc[i+1][GDP]
            bottoms[data_df.iloc[i+2]['Quarter']] = data_df.iloc[i+2][GDP]
    return bottoms.idxmin()
    ans = '2009q2'
    return data_df

get_recession_bottom()

'2009q2'

In [239]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    import numpy as np
    house_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    house_df.drop('RegionID', axis=1, inplace=True)
    house_df['State'] = house_df['State'].map(lambda s: states[s])
    house_df.set_index(["State","RegionName"], drop=True, inplace=True)
    start_year = 2000
    end_year = 2016
    all_columns = house_df.columns
    final_df = pd.DataFrame()
    final_df[['Metro', 'CountyName']] = house_df[['Metro', 'CountyName']].copy()
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if month >= 10: 
                year_month = '{}-{}'.format(year, month)
            else:
                year_month = '{}-0{}'.format(year, month)
            if year_month in all_columns:
                quarter = np.max([0, month - 1]) // 3 + 1
                title = '{}q{}'.format(year, quarter)
                if title not in final_df.columns:
                    final_df[title] = house_df[year_month].copy()
                else:
                    final_df[title] = final_df[title] + house_df[year_month]
    return final_df

convert_housing_data_to_quarters()

,,Metro,CountyName,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
California,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles,621200.0,643400.0,662900.0,678500.0,699000.0,717300.0,735200.0,759100.0,...,1494100,1527200,1556600,1586400,1614500,1641800,1673200,1698100,1732400,1168100
Illinois,Chicago,Chicago,Cook,415200.0,430900.0,443600.0,456400.0,470800.0,485400.0,499200.0,511300.0,...,577900,587300,603800,603200,618100,624900,623700,618200,624600,424000
Pennsylvania,Philadelphia,Philadelphia,Philadelphia,159000.0,160900.0,162400.0,164100.0,166000.0,166600.0,168800.0,172600.0,...,341200,345900,347000,348600,353900,363700,366600,370300,380800,257400
Arizona,Phoenix,Phoenix,Maricopa,335500.0,343100.0,348000.0,352200.0,358800.0,364700.0,368100.0,372900.0,...,492800,496100,505500,514600,522500,537200,551500,563700,574300,390400
Nevada,Las Vegas,Las Vegas,Clark,397800.0,403100.0,406200.0,411000.0,418600.0,425200.0,430100.0,438400.0,...,510200,520200,526400,532500,544800,560300,571900,583800,591600,399900
California,San Diego,San Diego,San Diego,668700.0,703100.0,736300.0,768100.0,801600.0,828800.0,853500.0,875800.0,...,1440600,1467100,1489300,1510100,1536200,1559300,1576400,1588000,1608700,1079500
Texas,Dallas,Dallas-Fort Worth,Dallas,253400.0,251600.0,254600.0,263500.0,269200.0,267900.0,267200.0,272700.0,...,319900,326700,334600,341100,363400,385700,403800,421500,433800,298600
California,San Jose,San Jose,Santa Clara,1122800.0,1219700.0,1295600.0,1366500.0,1412000.0,1410600.0,1370400.0,1336700.0,...,2038200,2091100,2144800,2194300,2270200,2329200,2367400,2410800,2456800,1644400
Florida,Jacksonville,Jacksonville,Duval,265800.0,269100.0,275100.0,279300.0,283200.0,286800.0,291200.0,297200.0,...,362300,365200,369500,372500,380700,390400,396000,401900,411600,279800


In [251]:
from scipy.stats import ttest_ind
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    data_df = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    print(recession_start,  recession_bottom)
    data_df = data_df.loc[:,recession_start: recession_bottom]
    data_df['ratio'] = ((data_df[recession_start] - data_df[recession_bottom]) / data_df[recession_start]).copy()
    university_towns = get_list_of_university_towns().set_index(['State', 'RegionName']).dropna()
    university_data = pd.merge(data_df, university_towns, how='inner', left_index=True, right_index=True).dropna()
    non_university_data = pd.concat([data_df, university_data]).drop_duplicates(keep=False)
    t,p = ttest_ind(university_data['ratio'], non_university_data['ratio'])
    better = 'university town'
    if non_university_data['ratio'].mean() < university_data['ratio'].mean():
        better = 'non-university town'
    difference = False
    if p < 0.01:
        difference = True
    return difference, p, better
run_ttest()

2008q3 2009q2


(True, 0.0057998740968818558, 'non-university town')

In [222]:
ttest_ind?

In [201]:
university_towns = get_list_of_university_towns().set_index(['State', 'RegionName'])
university_data = pd.merge(data_df, university_towns, how='inner', left_index=True, right_index=True)

In [212]:
pd.concat([data_df, university_data]).drop_duplicates(keep=False)

,,Metro,CountyName,State,RegionName,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,New York,Queens,New York,New York,0.0,0.0,0.0,0.0,0.0,0.0,...,1546400,1568400,1584200,1596800,1622400,1671600,1718500,1748600,1774900,1174400
California,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles,California,Los Angeles,621200.0,643400.0,662900.0,678500.0,699000.0,717300.0,...,1494100,1527200,1556600,1586400,1614500,1641800,1673200,1698100,1732400,1168100
Illinois,Chicago,Chicago,Cook,Illinois,Chicago,415200.0,430900.0,443600.0,456400.0,470800.0,485400.0,...,577900,587300,603800,603200,618100,624900,623700,618200,624600,424000
Pennsylvania,Philadelphia,Philadelphia,Philadelphia,Pennsylvania,Philadelphia,159000.0,160900.0,162400.0,164100.0,166000.0,166600.0,...,341200,345900,347000,348600,353900,363700,366600,370300,380800,257400
Arizona,Phoenix,Phoenix,Maricopa,Arizona,Phoenix,335500.0,343100.0,348000.0,352200.0,358800.0,364700.0,...,492800,496100,505500,514600,522500,537200,551500,563700,574300,390400
California,San Jose,San Jose,Santa Clara,California,San Jose,1122800.0,1219700.0,1295600.0,1366500.0,1412000.0,1410600.0,...,2038200,2091100,2144800,2194300,2270200,2329200,2367400,2410800,2456800,1644400
Florida,Jacksonville,Jacksonville,Duval,Florida,Jacksonville,265800.0,269100.0,275100.0,279300.0,283200.0,286800.0,...,362300,365200,369500,372500,380700,390400,396000,401900,411600,279800
California,San Francisco,San Francisco,San Francisco,California,San Francisco,1291500.0,1393400.0,1450600.0,1479000.0,1482200.0,1488400.0,...,2780800,2863600,2906300,3002200,3182400,3285300,3316400,3365300,3357800,2212800
Michigan,Detroit,Detroit,Wayne,Michigan,Detroit,198500.0,204900.0,200300.0,201100.0,202500.0,205100.0,...,111900,111300,111400,108600,107500,111200,115100,113900,112400,75800


In [213]:
len(university_data)

269